# Comprensione, analisi e pulizia dei dati

Il file originale da oltre 16M di tweets è stato diviso in dataset più piccoli da 1M di righe circa ciascuno (motivi computazionali) per ognuna delle operazioni di pre-processing dei dati, ragione per cui vengono caricati dataset con nomi diversi, in genere contenenti un numero, e per cui c'è molto codice ridondante.

# Data loading

In [ ]:
import pandas as pd
dfc = pd.read_csv('.../en_tweets.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
dfc[:20]

# Language Detection

In [ ]:
#Abbiamo selezionato soltanto i tweets in lingua inglese
#su porzioni del dataset originale
from langdetect import detect
prova1=dfc[15000001:]

def det(x):
    try:
        lang = detect(x)
    except:
        lang = 'Other'
    return lang
prova1['lang']= prova1['text'].apply(det)

In [ ]:
prova1.head()

In [ ]:
#ciascuna di queste parti è stata salvata in un file diverso
prova1.to_csv('.../split15.csv', index = False)

In [ ]:
#filtraggio dei tweet in base alla lingua
#abbiamo considerato solo tweet in inglese
en1=prova1[prova1['lang']=='en']

In [ ]:
#di nuovo salvataggio del file
en1.to_csv('.../en15.csv', index = False) 

In [ ]:
#dopo aver ripetuto il codice una seconda volta,
#man mano abbiamo unito tutti i file fino ad ottenere un dataset unico
en1_2=pd.concat([en1, en2], axis=0)
en1_2.to_csv('.../sa1_2.csv', index = False)

# SENTIMENT ANALYSIS CON VADER

In [ ]:
import pandas as pd
d= pd.read_csv('.../en_tweets.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
#Abbiamo fatto la sentiment analysis su tutto il dataset 
#applicando il codice a 1M di tweets alla volta

#Sono state aggiunte al dataset 4 colonne: 'compound', 'pos', 'neu', 'neg'
analyzer= SentimentIntensityAnalyzer()
d1=d[1000001:2000000]
for row in d1:
    d1['compound']=[analyzer.polarity_scores(x)['compound'] for x in d1['text']]
    d1['neg']=[analyzer.polarity_scores(x)['neg'] for x in d1['text']]
    d1['neu']=[analyzer.polarity_scores(x)['neu'] for x in d1['text']]
    d1['pos']=[analyzer.polarity_scores(x)['pos'] for x in d1['text']]

In [ ]:
d1.head()

In [ ]:
#ciascuna di queste parti è stata salvata in un file diverso
d1.to_csv('.../sa1.csv', index = False) 

In [ ]:
#concatenazione dei file
# d2=d[2000001:3000000]
d1_2=pd.concat([d1, d2], axis=0)
d1_2.to_csv('.../sa1_2.csv', index = False) #salvataggio finale

# Drop delle colonne superflue

In [ ]:
import pandas as pd
df = pd.read_csv('.../sa4.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df

In [ ]:
drop1 = df.drop(columns= ['replies', 'likes', 'retweets', 'lang'])

In [ ]:
drop1.to_csv('.../drop4.csv', index = False) 

# Colonna 'date'

In [ ]:
import pandas as pd
prova = pd.read_csv('.../drop4.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
prova

In [ ]:
import pandas as pd
prova['date'] = pd.to_datetime(prova['date'])
prova = prova.set_index(prova['date'])
prova = prova.sort_index()
cut = prova['2019-03-25':]

In [ ]:
cut.to_csv('.../drop1.csv', index = False) 

# Comprensione dei dati

In [ ]:
import pandas as pd
df = pd.read_csv('.../df2019.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df

In [ ]:
#Visualizzazione dei dati 
import matplotlib.pyplot as plt
import seaborn as sns

sns.distplot(df['compound'], hist=True, kde=True, 
             bins=int(180/5), color = 'darkblue', 
             hist_kws={'edgecolor':'black'},
             kde_kws={'linewidth': 4})

In [ ]:
df.columns=['date', 'user','timestamp', 'text', 'compound', 'neg', 'neu', 'pos']

In [ ]:
df

In [ ]:
df3 = df[df.pos > 0.8]

In [ ]:
pd.set_option('display.max_colwidth',700)

In [ ]:
df3

In [ ]:
(~df['text'].str.contains('BitCoin|bitcoin|BTC|btc|Bitcoin|BITCOIN|COIN|coin|Crypto|crypto|Btc')).sum()

In [ ]:
df4 = df[df['text'].str.contains('BitCoin|bitcoin|BTC|btc|Bitcoin|BITCOIN|COIN|coin|Crypto|crypto|CRYPTO|Btc')]

In [ ]:
df4

In [ ]:
df4['compound'].value_counts()

# Pulizia del testo

In [ ]:
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
import regex as re
import contractions
import sys

In [ ]:
import nltk
nltk.download('stopwords')

def process(text):
    #Convertire a lower case
    text = text.lower()
    #Trasformare www.* oppure https?://* in URL
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',text)
    #Rimuovere @username 
    text = re.sub('@[^\s]+',' ',text)
    #Rimuovere spazi bianchi in più
    text = re.sub('[\s]+', ' ', text)
    #Sostituire #parola con parola
    text = re.sub(r'#([^\s]+)', r'\1', text)
    #trim
    text = text.strip('\'"')
    #Rimozione della punteggiatura
    for punctuation in string.punctuation: 
        text = text.replace(punctuation, ' ') 
    #tenere solo parole
    text = ''.join([i for i in text if not i.isdigit()])
    #tokenizzare
    words = nltk.tokenize.casual_tokenize(text)
    #Rimuovere stopwords
    stops = set(stopwords.words('english'))
    clean = [w for w in words if not w in stops]
    return ' '.join(clean)

df['clean'] = df['text'].apply(lambda text: process(text))
df.sample(3, random_state = 3)

In [ ]:
#Abbiamo eliminato la sentiment analysis fatta in precedenza per ripeterla sul testo pulito 
#ed ottenere una maggiore precisione
new = df.drop(columns= ['text', 'compound', 'neu', 'neg', 'pos'])

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer= SentimentIntensityAnalyzer()
for row in new:
    new['neg']=[analyzer.polarity_scores(x)['neg'] for x in new['clean']]
    new['neu']=[analyzer.polarity_scores(x)['neu'] for x in new['clean']]
    new['pos']=[analyzer.polarity_scores(x)['pos'] for x in new['clean']]
    new['compound']=[analyzer.polarity_scores(x)['compound'] for x in new['clean']]

In [ ]:
new

In [ ]:
new.to_csv('.../file7.csv', index = False) 

#### Eravamo consapevoli della necessità della pulizia del testo per la SA, ma il nostro intento era cercare di avere più valori possibili da comparare. Infatti abbiamo tentato anche di dare un'etichetta unica per ciascun tweet (pos, neg e neu) invece di un valore compreso tra 0 e 1 per "positive, negative e neutral". Come ci aspettavamo, la scelta dell'eticehtta unica era troppo approssimativa e spesso errata, al contrario dei singoli valori dopo la pulizia del testo.

# Annotazione dei tweets con 'bull' oppure 'bear'

In [ ]:
import pandas as pd
df = pd.read_csv('.../file10.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df = df.set_index(df['date'])
#df = df.sort_index()

In [ ]:
#Ciascun file in cui il dataset è stato tagliato è stato annotato manualmente osservando gli andamenti della
#valuta sul grafico in tempo reale del bitcoin.
cut = df[:'2019-10-31']
cut['value'] = 'bull'
cut

In [ ]:
cut1 = df['2019-11-01': '2019-11-21']
cut1['value'] = 'bear'
cut1

In [ ]:
cut2 = df['2019-10-18': '2019-10-31']
cut2['value'] = 'bull'
cut2

In [ ]:
cut3 = df['2019-10-04':]
cut3['value'] = 'bull'
cut3

In [ ]:
cut.reset_index(drop=True, inplace=True)
cut1.reset_index(drop=True, inplace=True)
#cut2.reset_index(drop=True, inplace=True)
#cut3.reset_index(drop=True, inplace=True)

In [ ]:
frames = [cut, cut1]
result = pd.concat(frames)

In [ ]:
result

In [ ]:
result.to_csv('.../Value10.csv', index = False) 

# Drop dei duplicati

In [ ]:
import pandas as pd
df = pd.read_csv('.../Value10.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df

In [ ]:
df.drop_duplicates(subset=[''], inplace=True)

In [ ]:
df

In [ ]:
df.to_csv('.../DropDuplicates.csv', index = False) 

# Osservazione e visualizzazione dei dati in base a 'bull' e 'bear'

In [ ]:
df.columns=['date', 'user','timestamp', 'clean', 'pos', 'neg', 'neu','compound', 'value']
df['value'].replace(to_replace="bull.+", value=1, regex=True, inplace=True)
df['value'].replace(to_replace="bear.+", value=0, regex=True, inplace=True)
df

In [ ]:
df1 = df[df.value == 'bull']
df2 = df[df.value == 'bear']

In [ ]:
df1

In [ ]:
df2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

In [ ]:
y1 = df1['neg']
y0 = df1['neu']
y2 = df1['pos']
trace1 = go.Histogram(
    x=y0, name='neu',
    opacity=0.75
)
trace2 = go.Histogram(
    x=y1, name = 'neg',
    opacity=0.75
)
trace3 = go.Histogram(
    x=y2, name = 'pos',
    opacity=0.75
)
data = [trace1, trace2, trace3]
layout = go.Layout(barmode='overlay', title='Distribution')
fig = go.Figure(data=data, layout=layout)
fig

In [ ]:
df1[df1.pos > 0]

In [ ]:
df1[df1.neg > 0]

In [ ]:
df1[df1.compound == 0]

In [ ]:
df2[df2.pos > 0.5]

In [ ]:
df2[df2.neg > 0.5]

In [ ]:
df2[df2.compound == 0]

# Colonna 'week'

In [ ]:
from pandas import Timestamp
weeks = [g for n, g in df.set_index('timestamp').groupby(pd.TimeGrouper('W'))]

In [ ]:
df.date = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df['date'].dt.week

In [ ]:
#Con la colonna 'week' abbiamo annotato ciascun tweet con la settimana dell'anno in cui è stato pubblicato
df['week'] = df['date'].dt.week

In [ ]:
df

In [ ]:
df['week'].plot()

In [ ]:
df.to_csv('.../WeekTOT.csv', index = False)

In [ ]:
#Visualizzazione della distribuzione settimanale dei dati
plt.figure(figsize=(10,5))
plt.xlabel("week")
plt.ylabel("Counts")
df.week.value_counts().plot(kind='bar')
plt.show()

# Eliminazione dei tweets rumorosi 

In [ ]:
import pandas as pd
df = pd.read_csv('.../Week19.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df

In [ ]:
df2 = df[df['clean'].str.contains('bitcoin|btc|crypto|eth|coin|value')]
df2

In [ ]:
df2.to_csv('.../df2.csv', index = False) 

In [ ]:
#Numero dei tweets per 'bull' e per 'bear' dopo aver filtrato i testi
df2['value'].value_counts(normalize = True)

# Creazione dataset a frequenza costante

In [ ]:
#file divisi per settimane
df1 = pd.read_csv('.../Week26.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)
df2 = pd.read_csv('.../Week28.csv', delimiter=',', skiprows=0, lineterminator='\n', low_memory=False)

In [ ]:
#concatenazione fino ad ottenere un file finale train + test
df12 = pd.concat([df1, df2], axis=0)

In [ ]:
#crazione del train bilanciato
df_train=df_final[(df_final['date']!='2019-05-06')&(df_final['date']!='2019-05-11')&(df_final['date']!='2019-06-04')&(df_final['date']!='2019-06-09')&(df_final['date']!='2019-06-26')&(df_final['date']!='2019-06-28')&(df_final['date']!='2019-07-11')&(df_final['date']!='2019-07-12')&(df_final['date']!='2

In [ ]:
#crazione del test bilanciato
df_test=df_final[(df_final['date']=='2019-05-06')|(df_final['date']=='2019-05-11')|(df_final['date']=='2019-06-04')|(df_final['date']=='2019-06-09')|(df_final['date']=='2019-06-26')|(df_final['date']=='2019-06-28')|(df_final['date']=='2019-07-11')|(df_final['date']=='2019-07-12')|(df_final['date']=='2019-07-17')|(df_final['date']=='2019-08-05')|(df_final['date']=='2019-08-09')|(df_final['date']=='2019-08-21')|(df_final['date']=='2019-08-23')|(df_final['date']=='2019-09-10')|(df_final['date']=='2019-09-13')|(df_final['date']=='2019-10-16')|(df_final['date']=='2019-10-20')|(df_final['date']=='2019-10-30')|(df_final['date']=='2019-11-03')]

In [ ]:
#salvataggio
df_finale.to_csv('.../TrainTestTOT.csv')
df_trai.to_csv('.../train.csv')
df_test.to_csv('.../test.csv')